### This Notepad will demostrate creation of Url summarizer using Google Gemini Model

import all the libraries

In [1]:
#imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

### Connecting to Google Gemini

Add Gemini Api Key to .env file

In [17]:
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")
api_base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
api_model = "gemini-2.0-flash"
# Check the key

if not gemini_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not gemini_api_key.startswith("AI"):
    print("An API key was found, but it doesn't start AI; please check you're using the right key - see troubleshooting notebook")
elif gemini_api_key.strip() != gemini_api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### Initialize ai

In [12]:
openai = OpenAI(api_key=gemini_api_key, base_url= api_base_url)

### Create a class for the website

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """Create a class website with the url provided """

        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content,'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


## Testing the container

In [ ]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

### Google Gemini models support both system prompts (instructions) and user prompts, but a system prompt is not strictly required for every request.

In [6]:
system_prompt = "You are an assistant who analyses the content of a website and provide a short summary. Ignore the test that might be navigation related "
system_prompt += "Provide the response in Markdown"

### User Prompt

In [7]:
def define_user_prompt(website):
    user_prompt = f"You are looking at the website titled {website.title}"
    user_prompt += "\n Prepare a short summary of the content of the website. Include any news or announcements made in the site. \n"
    user_prompt +=  website.text

    return user_prompt

### Testing the user Prompt

In [ ]:
user_prompt = define_user_prompt(Website("https://edwarddonner.com"))
print(user_prompt)

### Messages for the prompt

In [15]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": define_user_prompt(website)}
    ]

### Invoke the Gemini agent

In [18]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = api_model,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

### Display Summary

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

### Testing

In [19]:
display_summary("https://edwarddonner.com")

Edward Donner's website introduces Ed as a coder, LLM enthusiast, DJ, and amateur electronic music producer. He is the co-founder and CTO of Nebula.io, an AI company focused on talent discovery, and was previously the founder/CEO of untapt (acquired in 2021). The site also includes announcements for courses and briefings, such as "Connecting my courses – become an LLM expert and leader" (May 28, 2025), "2025 AI Executive Briefing" (May 18, 2025) and "The Complete Agentic AI Engineering Course" (April 21, 2025). The website has links to his social media and a newsletter subscription option.
